# Non-convex inverse problems project: Sparse PCA
##### Yago Aguado, M2 MASH

In [29]:
#Modules
import numpy as np
from random import sample
import cvxpy as cp


#### Question 1:
Let us begin by creating a function to simulate the problem we will be trying to solve (cf. question 2).

In [ ]:
# Question 1: code a function generating an instance of the problem

def simulation(k, m, n):
    # Sample the lists of non-zero indices
    l = range(n) #indices are those in python
    s1 = sample(l, k)
    s2 = sample(l, k)
    
    # Create the vectors forming x
    u = np.zeros(n)
    v = np.zeros(n)
    
    c1 = np.random.normal(0, 1, k)
    c2 = np.random.normal(0, 1, k)
    
    for i in range(k):
        u[s1[i]] = c1[i]
        v[s2[i]] = c2[i]
    
    # Compute x
    X = np.outer(u, v)
    
    # Generate the random matrices A_i
    lA = []
    ly = []
    for i in range(m):
        A = np.random.uniform(0, 1, (n,n))
        y = np.vdot(A, X) #Frobenius product
        lA.append(A)
        ly.append(y)
    
    return X, s1, s2, lA, ly
        
    

In [31]:
# Question 3: let us implement a solver for the convex approach

def solver(lbda, lA, ly, n, m):
    # Define the problem
    X = cp.Variable((n, n))
    constraints = [np.vdot(lA[i], X) == ly[i] for i in range(m)]
    objective = cp.Minimize(cp.norm(X,"nuc") + lbda * cp.pnorm(X, 1))
    problem = cp.Problem(objective,constraints)
    
    
    # Solve it
    problem.solve(solver=cp.SCS,verbose=True)
    return X.value

def error(X, s1, s2, z):
    n = np.shape(X)[0]
    min = np.abs(X[s1[0], s2[0]])
    for i in s1:
        for j in s2:
            if np.abs(X[i, j])< min:
                min = np.abs(X[i, j])
    
    err = 0
    for i in range(n):
        for j in range(n):
            if i in s1 and j in s2:
                if z[i, j] < min/10:
                    err += 1
            else:
                if z[i, j] > min/10:
                    err += 1
    
    return err